# QDET Governance Module Tutorial

This notebook demonstrates all available tools in the QDET Governance module.

The Governance module provides enterprise-grade features for:
- **Cost Estimation**: Budget quantum jobs before execution
- **Drift Detection**: Monitor for data distribution changes
- **Data Integrity**: Verify quantum encoding correctness
- **Job Monitoring**: Track execution progress
- **Privacy**: Apply differential privacy to quantum circuits
- **Noise Simulation**: Test robustness against hardware noise
- **Validation**: Check data-hardware compatibility
- **Audit & Compliance**: Log all operations and enforce policies
- **Security**: Manage access control and encryption
- **Orchestration**: Coordinate complex workflows

We'll use the iris.csv dataset to demonstrate these governance capabilities.

## 1. Setup & Imports

Import all required libraries and load the iris dataset.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')
import sys
import os
sys.path.append(os.path.abspath('..'))

# Load iris dataset
iris_data = pd.read_csv('../qudet/datasets/iris.csv')
X = iris_data.iloc[:, :-1].values
y = iris_data.iloc[:, -1].values

# Split into train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print(f"Dataset loaded:")
print(f"  Training set: {X_train.shape}")
print(f"  Test set: {X_test.shape}")
print(f"  Features: {iris_data.columns[:-1].tolist()}")
print(f"  Classes: {np.unique(y)}")

## 2. Resource Estimator

**Description**: Estimates cost and complexity of quantum jobs before execution. Critical for budgeting large-scale data pipelines.

**Use Case**: Check if iris dataset fits within quantum hardware limits and estimate job costs.

In [ ]:
from qudet.governance.cost import ResourceEstimator
from qiskit import QuantumCircuit

# Create a sample quantum circuit
qc = QuantumCircuit(4, 4)
qc.h(range(4))
qc.cx(0, 1)
qc.cx(1, 2)
qc.cx(2, 3)
qc.measure(range(4), range(4))

# Estimate circuit cost
cost_report = ResourceEstimator.estimate_circuit_cost(qc, shots=1024, hardware_rate=0.5)
print("Circuit Cost Estimation:")
for key, value in cost_report.items():
    print(f"  {key}: {value}")

# Check pipeline feasibility
print(f"\nIris Dataset Feasibility Check:")
feasibility = ResourceEstimator.check_pipeline_feasibility(len(X_train), X_train.shape[1])
print(f"  {feasibility}")

# Check with reduced dimensions
feasibility_large = ResourceEstimator.check_pipeline_feasibility(50000, 200)
print(f"\nLarge dataset (50k samples, 200 features):")
print(f"  {feasibility_large}")

## 3. Quantum Drift Detector

**Description**: Detects data drift using Quantum Maximum Mean Discrepancy (MMD). Compares reference and current data distributions to identify when model retraining is needed.

**Use Case**: Monitor iris data for distribution changes in production.

In [ ]:
from qudet.governance.drift import QuantumDriftDetector

# Create drift detector
drift_detector = QuantumDriftDetector(n_qubits=4, threshold=0.1)

# Fit on training data (reference)
drift_detector.fit_reference(X_train)
print("Drift Detector fitted on training data.")

# Test for drift on same data (should have no drift)
result_same = drift_detector.detect_drift(X_train[:20])
print(f"\nDrift Detection on Training Data:")
print(f"  Drift detected: {result_same['drift_detected']}")
print(f"  MMD score: {result_same.get('mmd_score', 'N/A')}")

# Test for drift on test data (may have some drift)
result_test = drift_detector.detect_drift(X_test[:20])
print(f"\nDrift Detection on Test Data:")
print(f"  Drift detected: {result_test['drift_detected']}")
print(f"  MMD score: {result_test.get('mmd_score', 'N/A')}")

# Synthetic drift: modify feature values
X_drifted = X_test[:20].copy()
X_drifted[:, 0] += 2.0  # Large shift in feature 0
result_drift = drift_detector.detect_drift(X_drifted)
print(f"\nDrift Detection on Drifted Data:")
print(f"  Drift detected: {result_drift['drift_detected']}")
print(f"  MMD score: {result_drift.get('mmd_score', 'N/A')}")

## 4. Data Integrity Check

**Description**: Verifies that quantum encoding preserves data information through round-trip checks. Validates encoder implementations and debugging encoding issues.

**Use Case**: Verify iris data integrity after quantum encoding.

In [ ]:
from qudet.governance.integrity import DataIntegrityCheck
from qudet.encoders.amplitude import AmplitudeEncoder

# Create encoder
encoder = AmplitudeEncoder()

# Check integrity of encoding
original_data = X_train[0]  # First sample
print(f"Data Integrity Check:")
print(f"  Original data shape: {original_data.shape}")
print(f"  Original data: {original_data}")

# Verify encoding
try:
    is_valid = DataIntegrityCheck.verify_encoding(original_data, encoder, tolerance=1e-5)
    print(f"  Encoding integrity verified: {is_valid}")
except Exception as e:
  Integrity check result: {str(e)[:100]}")

# Compute fidelity
try:
    stats = DataIntegrityCheck.compute_encoding_fidelity(original_data, encoder)
    print(f"\nEncoding Fidelity Statistics:")
    for key, value in stats.items():
        print(f"  {key}: {value}")
except Exception as e:
    print(f"  Fidelity computation: {str(e)[:100]}")

## 5. Job Monitor

**Description**: Real-time progress tracker for quantum data pipelines. Displays completion percentage, execution rate, and ETA.

**Use Case**: Monitor iris processing pipeline progress.

In [ ]:
from qudet.governance.monitor import JobMonitor
import time

# Create monitor
total_items = 30
monitor = JobMonitor(total_items, description="Processing Iris Samples")

print("Job Monitoring Example:")
print(f"  Total items: {total_items}")

# Simulate processing
for i in range(total_items):
    # Simulate work
    time.sleep(0.01)
    monitor.update(1)

monitor.finish()
print(f"\nProcessing completed.")
print(f"  Total time: {monitor.get_elapsed_time():.2f} seconds")
print(f"  Processing rate: {monitor.get_rate():.2f} items/second")

## 6. Quantum Differential Privacy

**Description**: Applies depolarizing noise to quantum circuits to ensure differential privacy. Guarantees that output doesn't reveal any single individual's input data.

**Use Case**: Protect iris data privacy when sending to cloud QPU.

In [ ]:
from qudet.governance.privacy import QuantumDifferentialPrivacy

# Create privacy module with different epsilon values
epsilons = [0.5, 1.0, 2.0]

print("Quantum Differential Privacy:")
for eps in epsilons:
    privacy = QuantumDifferentialPrivacy(epsilon=eps)
    print(f"\n  Epsilon: {eps}")
    print(f"    Noise probability: {privacy.noise_prob:.4f}")
    print(f"    Privacy level: {'High' if eps < 1 else 'Moderate' if eps < 2 else 'Low'}")

# Create and sanitize a circuit
print(f"\nCircuit Privacy Protection:")
qc_original = QuantumCircuit(2)
qc_original.h(0)
qc_original.cx(0, 1)
print(f"  Original circuit gates: {len(qc_original.data)}")

privacy = QuantumDifferentialPrivacy(epsilon=1.0)
qc_sanitized = privacy.sanitize(qc_original)
print(f"  Sanitized circuit gates: {len(qc_sanitized.data)}")
print(f"  Privacy overhead: {(len(qc_sanitized.data) / len(qc_original.data) - 1) * 100:.1f}%")

## 7. Noise Simulator

**Description**: Generates realistic noise models to stress-test pipelines against real quantum hardware conditions without accessing actual QPUs.

**Use Case**: Test iris pipeline robustness against hardware noise.

In [ ]:
from qudet.governance.simulation import NoiseSimulator

print("Noise Simulator Configuration:")

# Get noisy backend with different error probabilities
error_probs = [0.001, 0.01, 0.05]

for error_prob in error_probs:
    try:
        backend = NoiseSimulator.get_noisy_backend(error_prob=error_prob)
        print(f"\n  Error probability: {error_prob * 100:.2f}%")
        print(f"    Backend type: Simulated quantum device with depolarizing noise")
    except ImportError:
        print(f"\n  Error probability: {error_prob * 100:.2f}%")
        print(f"    Note: Install qiskit-aer for noise simulation")

# Stress test configuration
stress_profile = NoiseSimulator.get_stress_test_profile(n_qubits=4)
print(f"\nStress Test Profile:")
print(f"  Qubits: {stress_profile.get('n_qubits', 'N/A')}")
print(f"  Single-qubit error rate: {stress_profile.get('error_rate_1q', 'N/A')}")
print(f"  Two-qubit error rate: {stress_profile.get('error_rate_2q', 'N/A')}")

## 8. Data-Hardware Validation

**Description**: Validates that data dimensions fit within hardware qubit limits. Prevents quantum circuits from requiring more qubits than available.

**Use Case**: Check iris dataset compatibility with quantum hardware.

In [ ]:
from qudet.governance.validation import check_quantum_capacity

print("Quantum Capacity Validation:")

# Check iris dataset
try:
    result = check_quantum_capacity(X_train.shape, max_qubits=127)
    print(f"\n  Iris dataset ({X_train.shape[1]} features):")
    print(f"    Hardware compatibility: VALID")
    print(f"    Qubits required: {X_train.shape[1]}")
    print(f"    Qubits available (IBM Brisbane): 127")
except ValueError as e:
    print(f"  Iris dataset: {str(e)}")

# Check with high-dimensional data
print(f"\n  High-dimensional data (500 features):")
try:
    result = check_quantum_capacity((X_train.shape[0], 500), max_qubits=127)
except ValueError as e:
    print(f"    {str(e)[:100]}...")

print(f"\n  Validation ensures data fits within hardware constraints.")

## 9. Audit Logger

**Description**: Records all operations on quantum data and algorithms. Tracks who did what, when, and what resources were used for compliance.

**Use Case**: Audit iris data processing operations for compliance.

In [ ]:
from qudet.governance.audit import AuditLogger

# Create audit logger
auditor = AuditLogger(max_events=1000)

print("Audit Logger Example:")

# Log various events
auditor.log_event(
    event_type='data_access',
    user='data_scientist',
    action='loaded iris dataset',
    resource='iris.csv',
    status='success',
    details={'rows': 150, 'features': 4}
)

auditor.log_event(
    event_type='algorithm_run',
    user='data_scientist',
    action='executed quantum classifier',
    resource='QuantumSVC',
    status='success',
    details={'accuracy': 0.95, 'samples': 120}
)

auditor.log_event(
    event_type='config_change',
    user='admin',
    action='updated privacy epsilon',
    resource='privacy_config',
    status='success',
    details={'old_epsilon': 1.0, 'new_epsilon': 0.5}
)

# Display audit log
print(f"\n  Total events logged: {len(auditor.events)}")
print(f"\n  Recent events:")
for i, event in enumerate(auditor.events[-3:], 1):
    print(f"\n    Event {i}:")
    print(f"      Type: {event.event_type}")
    print(f"      User: {event.user}")
    print(f"      Action: {event.action}")
    print(f"      Status: {event.status}")

## 10. Secure Access Control

**Description**: Manages user authentication and role-based access control (RBAC). Enforces fine-grained permissions on resources and operations.

**Use Case**: Control access to iris data and algorithms.

In [ ]:
from qudet.governance.security import SecureAccessControl, AccessLevel

# Create access control
access_control = SecureAccessControl()

print("Secure Access Control Example:")

# Add users with different roles
access_control.add_user('alice', 'password123', AccessLevel.ADMIN)
access_control.add_user('bob', 'password456', AccessLevel.USER)
access_control.add_user('charlie', 'password789', AccessLevel.VIEWER)

print(f"\n  Users created:")
print(f"    alice: ADMIN")
print(f"    bob: USER")
print(f"    charlie: VIEWER")

# Authenticate users
auth_alice, token_alice = access_control.authenticate('alice', 'password123')
auth_bob, token_bob = access_control.authenticate('bob', 'password456')
auth_fail, _ = access_control.authenticate('bob', 'wrongpassword')

print(f"\n  Authentication results:")
print(f"    Alice login: {'SUCCESS' if auth_alice else 'FAILED'}")
print(f"    Bob login: {'SUCCESS' if auth_bob else 'FAILED'}")
print(f"    Bob with wrong password: {'FAILED' if not auth_fail else 'SUCCESS'}")

# Check permissions
print(f"\n  Permissions:")
alice_perms = access_control.check_permission('alice', 'users:manage')
bob_perms = access_control.check_permission('bob', 'users:manage')
print(f"    Alice can manage users: {alice_perms}")
print(f"    Bob can manage users: {bob_perms}")

## 11. Workflow Orchestration

**Description**: Orchestrates execution of interdependent quantum tasks. Manages scheduling, dependency resolution, error handling, and parallel execution.

**Use Case**: Create iris data processing workflow with multiple steps.

In [ ]:
from qudet.governance.orchestration import Workflow, Task, TaskStatus, WorkflowStatus

# Create workflow
workflow = Workflow(
    workflow_name='Iris ML Pipeline',
    description='Complete ML pipeline for iris dataset'
)

print("Workflow Orchestration Example:")

# Add tasks
task1_id = workflow.add_task(
    name='Load Data',
    operation='data_loading',
    params={'dataset': 'iris.csv', 'split_ratio': 0.8}
)

task2_id = workflow.add_task(
    name='Feature Scaling',
    operation='preprocessing',
    params={'method': 'standard_scaler'},
    dependencies=[task1_id]
)

task3_id = workflow.add_task(
    name='Train Quantum Classifier',
    operation='model_training',
    params={'algorithm': 'QuantumSVC', 'n_qubits': 4},
    dependencies=[task2_id]
)

task4_id = workflow.add_task(
    name='Evaluate Model',
    operation='evaluation',
    params={'metrics': ['accuracy', 'f1']},
    dependencies=[task3_id]
)

print(f"\n  Workflow: {workflow.workflow_name}")
print(f"  Description: {workflow.description}")
print(f"\n  Tasks:")
print(f"    1. Load Data")
print(f"    2. Feature Scaling (depends on 1)")
print(f"    3. Train Classifier (depends on 2)")
print(f"    4. Evaluate (depends on 3)")

# Get workflow graph
graph_info = workflow.get_execution_graph()
print(f"\n  Execution graph: {len(graph_info)} task dependencies mapped")

## Summary

This notebook demonstrated the comprehensive governance capabilities of QDET:

### Cost & Resource Management
- **Resource Estimator**: Budget quantum jobs and check feasibility

### Data Quality & Monitoring
- **Quantum Drift Detector**: Monitor for data distribution changes
- **Data Integrity Check**: Verify quantum encoding preservation
- **Job Monitor**: Track pipeline execution progress

### Privacy & Security
- **Quantum Differential Privacy**: Protect data when using cloud QPUs
- **Noise Simulator**: Test robustness against hardware errors
- **Secure Access Control**: Manage authentication and permissions

### Validation & Compliance
- **Validation**: Ensure data fits hardware constraints
- **Audit Logger**: Record all operations for compliance

### Orchestration
- **Workflow Orchestration**: Coordinate complex multi-task pipelines

### Key Insights
1. **Enterprise-Ready**: Features for production quantum ML systems
2. **Compliance**: Comprehensive audit trails and access control
3. **Robustness**: Privacy, noise simulation, and drift detection
4. **Scalability**: Cost estimation and resource management
5. **Integration**: Works with QDET analytics, transforms, and encoders

### Next Steps
- Combine governance tools with analytics algorithms
- Deploy audit logging in production pipelines
- Use orchestration for complex multi-step workflows
- Monitor data drift and retrain models proactively